# CS7643 - Final Project

In [16]:
# built-in
# public
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
# private
from config import Config, InstrConfig
from src.utils import dataloader, helpers
from src.models.instructions_generator_model import InstructionsGeneratorModel
from src.trainer import instructions_generator_trainer
from src.models.imitation_learning_model import ImitationLearningModel
from src.trainer import imitation_learning_trainer

%load_ext autoreload 
%autoreload 2
%config Completer.use_jedi = False

# Initialization

In [17]:
instr_config = InstrConfig()
helpers.set_seed(instr_config.random_seed)

# Data

## Load Data

In [18]:
# train size 202831
# unique number of instructions 35921
data = dataloader.load_pkl(workdir=instr_config.DATA_PATH)

In [19]:
train_states, train_inventories, train_actions, train_goals, train_instructions, all_instructions = data

In [20]:
# remove invalid sample where train instruction is None
invalid_index = set([i for i, _ in enumerate(train_instructions) if not _])
print(len(invalid_index))

34


In [21]:
valid_index = [i for i, _ in enumerate(train_instructions) if _]
print(len(valid_index))
train_states = np.array(train_states)[valid_index].tolist()
train_inventories = np.array(train_inventories)[valid_index].tolist()
train_actions = np.array(train_actions)[valid_index].tolist()
train_instructions = np.array(train_instructions)[valid_index].tolist()

202797


## Build Vocab

In [22]:
vocab, vocab_weights = dataloader.generate_vocab(
    all_instructions, instr_config.device, workdir=instr_config.DATA_PATH)

# vocab, vocab_weights = dataloader.generate_vocab(
#     all_instructions, instr_config.device, workdir=instr_config.DATA_PATH, cache='/usr/local/google/home/billzhou/Documents/glove')

Total vocabulary size: 212


## Generate Dataset

In [23]:
dataset = dataloader.CraftingDataset(
  instr_config.embeded_dim,
  train_states,
  train_inventories,
  train_actions,
  train_goals,
  train_instructions,
  vocab)

# dataset = dataloader.CraftingDataset(
#   instr_config.embeded_dim,
#   train_states,
#   train_inventories,
#   train_actions,
#   train_goals,
#   train_instructions,
#   vocab,
#   cache='/usr/local/google/home/billzhou/Documents/glove')

embedding loaded
one hot loaded
actions loaded
goals loaded
done loading dataset


In [9]:
min([len(d[-1]) for d in dataset])

3

In [10]:
instr_config.dataset_size = len(dataset)

## Split Dataset

In [24]:
indices = list(range(instr_config.dataset_size))
split = int(np.floor(instr_config.validation_split * instr_config.dataset_size))
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

AttributeError: 'InstrConfig' object has no attribute 'dataset_size'

## Initialize Data Loader

In [ ]:
train_data_loader = DataLoader(
  dataset,
  batch_size=instr_config.batch_size,
  num_workers=0,
  pin_memory=True,
  sampler=train_sampler,
  collate_fn=dataloader.collate_fn)

validation_data_loader = DataLoader(
  dataset,
  batch_size=instr_config.batch_size,
  num_workers=0,
  pin_memory=True,
  sampler=valid_sampler,
  collate_fn=dataloader.collate_fn)

# IL + LSTM Training

## Instructions Generator (LSTM)

### Setup Instructions Generator Training

In [14]:
model = InstructionsGeneratorModel(
    instr_config.device
    , vocab
    , instr_config.embeded_dim
    , vocab_weights
).to(instr_config.device)
train = instructions_generator_trainer.train
validate = instructions_generator_trainer.validate

In [15]:
# CE Loss
criterion = torch.nn.CrossEntropyLoss()
# Adam
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=instr_config.learning_rate)
# Log
writer = SummaryWriter() if instr_config.summary_writer else None

### GO

In [15]:
best_valid_loss = float('inf')
valid_epoch, best_epoch = 0, None

for epoch in range(instr_config.epochs):
    # train
    loss, bleu, tk_acc = train(
        instr_config.device,
        epoch,
        train_data_loader,
        model,
        optimizer,
        criterion,
        parameters,
        vocab,
        summary_writer=writer)
    print('Overall Epoch: %d, train loss: %.3f, train bleu: %.3f, train token acc: %.3f' % (epoch, loss, bleu, tk_acc))
    # valid
    loss, bleu, tk_acc = validate(
        instr_config.device,
        epoch,
        validation_data_loader,
        model,
        criterion,
        vocab,
        summary_writer=writer)
    print('Overall Epoch: %d, valid loss: %.3f, valid bleu: %.3f, valid token acc: %.3f' % (epoch, loss, bleu, tk_acc))
    # early stopping
    if loss <= best_valid_loss:
        best_valid_loss = loss
        valid_epoch, best_epoch = 0, epoch
        torch.save(model.state_dict(), instr_config.SAVE_PATH)
        print('Best Epoch: %d, best valid loss: %.3f' % (best_epoch, best_valid_loss))
        print('Trained model saved at ', instr_config.SAVE_PATH)
    else:
        valid_epoch += 1
        if valid_epoch >= instr_config.valid_patience:
            break

Epoch: 0, train loss: 2.605, train bleu: 0.589, train token acc: 0.720: 100%|██████████| 2535/2535 [04:53<00:00,  8.65it/s]


Overall Epoch: 0, train loss: 2.900, train bleu: 0.530, train token acc: 0.684


Epoch: 0, valid loss: 4.509, valid bleu: 0.426, valid token acc: 0.557: 100%|██████████| 634/634 [00:54<00:00, 11.58it/s]


Overall Epoch: 0, valid loss: 4.593, valid bleu: 0.477, valid token acc: 0.592
Best Epoch: 0, best valid loss: 4.593
Trained model saved at  /google/src/cloud/billzhou/ask_your_human/google3/experimental/users/billzhou/Ask-Your-Humans/res/cpts/instr_gen.pt


Epoch: 1, train loss: 2.450, train bleu: 0.590, train token acc: 0.728: 100%|██████████| 2535/2535 [04:55<00:00,  8.58it/s]


...

Overall Epoch: 3, valid loss: 5.289, valid bleu: 0.489, valid token acc: 0.598


Epoch: 4, train loss: 2.065, train bleu: 0.635, train token acc: 0.777: 100%|██████████| 2535/2535 [04:58<00:00,  8.50it/s]


Overall Epoch: 4, train loss: 2.128, train bleu: 0.616, train token acc: 0.764


Epoch: 4, valid loss: 4.621, valid bleu: 0.412, valid token acc: 0.588: 100%|██████████| 634/634 [00:53<00:00, 11.93it/s]

Overall Epoch: 4, valid loss: 5.338, valid bleu: 0.491, valid token acc: 0.603


## Imitation Learning + LSTM

### Set up

In [25]:
lstm_model = InstructionsGeneratorModel(
    instr_config.device
    , vocab
    , instr_config.embeded_dim
    , vocab_weights
).to(instr_config.device)
il_model = ImitationLearningModel(instr_config.embeded_dim).to(instr_config.device)

train = imitation_learning_trainer.train
validate = imitation_learning_trainer.validate

In [26]:
# CE Loss
il_criterion = torch.nn.CrossEntropyLoss()
lstm_criterion = torch.nn.CrossEntropyLoss()
# Adam
il_parameters = filter(lambda p: p.requires_grad, il_model.parameters())
il_optimizer = torch.optim.Adam(il_parameters, lr=instr_config.learning_rate)
lstm_parameters = filter(lambda p: p.requires_grad, lstm_model.parameters())
lstm_optimizer = torch.optim.Adam(lstm_parameters, lr=instr_config.learning_rate)
# Log
writer = SummaryWriter() if instr_config.summary_writer else None

### Train IL + LSTM

In [17]:
best_valid_loss = float('inf')
valid_epoch, best_epoch = 0, None

for epoch in range(instr_config.epochs):
    # train
    action_loss, lstm_loss, acc, bleu, tk_acc = train(
        instr_config.device,
        epoch,
        train_data_loader,
        il_model,
        lstm_model,
        il_optimizer,
        lstm_optimizer,
        il_criterion,
        lstm_criterion,
        il_parameters,
        lstm_parameters,
        vocab,
        summary_writer=writer)
    print('Overall Epoch: %d, train action loss: %.3f, train lang loss: %.3f, train action acc: %.3f, train bleu: %.3f, train token acc: %.3f' % (epoch, action_loss, lstm_loss, acc, bleu, tk_acc))
    # valid
    action_loss, lstm_loss, acc, bleu, tk_acc = validate(
        instr_config.device,
        epoch,
        validation_data_loader,
        il_model,
        lstm_model,
        il_criterion,
        lstm_criterion,
        vocab,
        summary_writer=writer)
    print('Overall Epoch: %d, valid action loss: %.3f, valid lang loss: %.3f, valid action acc: %.3f, valid bleu: %.3f, valid token acc: %.3f' % (epoch, action_loss, lstm_loss, acc, bleu, tk_acc))
    # early stopping
    if action_loss <= best_valid_loss:
        best_valid_loss = action_loss
        valid_epoch, best_epoch = 0, epoch
        torch.save(il_model.state_dict(), instr_config.SAVE_PATH)
        print('Best Epoch: %d, best valid loss: %.3f' % (best_epoch, best_valid_loss))
        print('Trained model saved at ', instr_config.SAVE_PATH)
    else:
        valid_epoch += 1
        if valid_epoch >= instr_config.valid_patience:
            break

Epoch: 0, train action loss: 0.861, train lang loss: 2.711, train action acc: 0.694, train bleu: 0.557, train token acc: 0.719: 100%|██████████| 2535/2535 [04:57<00:00,  8.52it/s]


Overall Epoch: 0, train action loss: 0.926, train lang loss: 3.130, train action acc: 0.673, train bleu: 0.512, train token acc: 0.643


Epoch: 0, valid action loss: 1.102, valid lang loss: 3.651, valid acc: 0.596, valid bleu: 0.376, valid token acc: 0.487: 100%|██████████| 634/634 [00:55<00:00, 11.42it/s]


Overall Epoch: 0, valid action loss: 0.952, valid lang loss: 3.281, valid action acc: 0.667, valid bleu: 0.493, valid token acc: 0.610
Best Epoch: 0, best valid loss: 0.952
Trained model saved at  /google/src/cloud/billzhou/ask_your_human/google3/experimental/users/billzhou/Ask-Your-Humans/res/cpts/instr_gen.pt


Epoch: 1, train action loss: 0.764, train lang loss: 2.608, train action acc: 0.758, train bleu: 0.593, train token acc: 0.734: 100%|██████████| 2535/2535 [05:00<00:00,  8.44it/s]


...

Overall Epoch: 18, valid action loss: 0.855, valid lang loss: 3.139, valid action acc: 0.708, valid bleu: 0.505, valid token acc: 0.615
Best Epoch: 18, best valid loss: 0.855
Trained model saved at  /google/src/cloud/billzhou/ask_your_human/google3/experimental/users/billzhou/Ask-Your-Humans/res/cpts/instr_gen.pt


Epoch: 19, train action loss: 1.017, train lang loss: 3.273, train action acc: 0.597, train bleu: 0.460, train token acc: 0.582: 100%|██████████| 2535/2535 [05:00<00:00,  8.43it/s]


Overall Epoch: 19, train action loss: 0.696, train lang loss: 2.858, train action acc: 0.763, train bleu: 0.544, train token acc: 0.673


Epoch: 19, valid action loss: 1.007, valid lang loss: 3.673, valid acc: 0.574, valid bleu: 0.450, valid token acc: 0.553: 100%|██████████| 634/634 [00:55<00:00, 11.50it/s]

Overall Epoch: 19, valid action loss: 0.859, valid lang loss: 3.157, valid action acc: 0.711, valid bleu: 0.501, valid token acc: 0.614


### Train IL Only

In [ ]:
best_valid_loss = float('inf')
valid_epoch, best_epoch = 0, None

for epoch in range(instr_config.epochs):
    # train
    action_loss, lstm_loss, acc, bleu, tk_acc = train(
        instr_config.device,
        epoch,
        train_data_loader,
        il_model,
        lstm_model,
        il_optimizer,
        lstm_optimizer,
        il_criterion,
        lstm_criterion,
        il_parameters,
        lstm_parameters,
        vocab,
        use_generative_language=False,
        summary_writer=writer)
    print('Overall Epoch: %d, train action loss: %.3f, train action acc: %.3f' % (epoch, action_loss, acc))
    # valid
    action_loss, lstm_loss, acc, bleu, tk_acc = validate(
        instr_config.device,
        epoch,
        validation_data_loader,
        il_model,
        lstm_model,
        il_criterion,
        lstm_criterion,
        vocab,
        use_generative_language=False,
        summary_writer=writer)
    print('Overall Epoch: %d, valid action loss: %.3f, valid action acc: %.3f' % (epoch, action_loss, acc))
    # early stopping
    if action_loss <= best_valid_loss:
        best_valid_loss = action_loss
        valid_epoch, best_epoch = 0, epoch
        torch.save(il_model.state_dict(), instr_config.SAVE_PATH)
        print('Best Epoch: %d, best valid loss: %.3f' % (best_epoch, best_valid_loss))
        print('Trained model saved at ', instr_config.SAVE_PATH)
    else:
        valid_epoch += 1
        if valid_epoch >= instr_config.valid_patience:
            break

Epoch: 0, train action loss: 1.263 train action acc: 0.581: 100%|██████████| 2535/2535 [00:29<00:00, 86.40it/s]


Overall Epoch: 0, train action loss: 1.278, train action acc: 0.523


Epoch: 0, valid action loss: 1.414 valid action acc: 0.532: 100%|██████████| 634/634 [00:05<00:00, 116.60it/s]


Overall Epoch: 0, valid action loss: 1.225, valid action acc: 0.551
Best Epoch: 0, best valid loss: 1.225
Trained model saved at  /google/src/cloud/billzhou/ask_your_human/google3/experimental/users/billzhou/Ask-Your-Humans/res/cpts/instr_gen.pt


Epoch: 1, train action loss: 1.300 train action acc: 0.516: 100%|██████████| 2535/2535 [00:29<00:00, 86.76it/s]


...

Epoch: 13, train action loss: 0.923 train action acc: 0.677: 100%|██████████| 2535/2535 [00:35<00:00, 70.94it/s]


Overall Epoch: 13, train action loss: 1.009, train action acc: 0.643


Epoch: 13, valid action loss: 1.195 valid action acc: 0.617: 100%|██████████| 634/634 [00:06<00:00, 91.38it/s] 


Overall Epoch: 13, valid action loss: 1.077, valid action acc: 0.620


Epoch: 14, train action loss: 1.015 train action acc: 0.688:  51%|█████     | 1284/2535 [00:17<00:17, 69.56it/s]